In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_left = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')
df_left

In [ ]:
df = pd.DataFrame()

cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    df_left[category] = df_left[category] * cat_mtpl[category]
    
df['score'] = (df_left[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) ).astype(float)
df['score'] = df['score']/df['score'].max()
df['text'] = df_left['comment_text']

In [ ]:
df

In [ ]:
df_train = pd.read_csv('/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv')
df_train

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

stemmer = SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()

def drop_suffix(sent):
    text = sent
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')])
    #text = ' '.join([word for word in text.split(' ') if word not in stop])
    return text

In [ ]:
import re
def replaceURL(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

def replaceAbbrev(text):
    text = text.replace("what's", "what is ").replace("\'ve", " have ").replace(r"can't", "cannot ").replace(r"n't", " not ").replace(r"i'm", "i am ").replace(r"\'re", " are ").replace(r"\'d", " would ").replace(r"\'ll", " will ").replace(r"\'scuse", " excuse ").replace(r"\'s", " ")
    return text

In [ ]:
def removeUnicode(text):
    text = re.sub(r'(\\u[0-9A-Fa-f]+)',r' ', text)       
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    text=re.sub(r'([a-zA-Z])\1{2,}\b',r'\1\1',text)
    text=re.sub(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1',text)
    text=re.sub(r'[ ]{2,}',' ',text)
    text = re.sub('@[^\s]+','atUser',text)
    text = re.sub(r'(fuckfuck)','fuck fuck ',text)
    text = re.sub(r'(f+)( *)([u|*|_]+)( *)([c|*|_]+)( *)(k)+','fuck',text)
    text = re.sub(r'(h+)(a+)(h+)(a+)','ha ha ',text)
    text = re.sub(r'(s+ *h+ *[i|!]+ *t+)','shit',text)
    text = re.sub(r'\b(n+)(i+)(g+)(a+)\b','nigga',text)
    text = re.sub(r'\b(n+)([i|!]+)(g+)(e+)(r+)\b','nigger',text)
    text = re.sub(r'\b(d+)(o+)(u+)(c+)(h+)(e+)( *)(b+)(a+)(g+)\b','douchebag',text)
    text = re.sub(r'([a|@][$|s][s|$])','ass',text)
    text = re.sub(r'(\bfuk\b)','fuck',text)
    text = re.sub(r"(^|\W)\d+", " ", text)
    text = text.replace("5","s")
    text = text.replace("1","i")
    text = text.replace("0","o")
    text=re.sub(r'([!])\1\1{2,}',r' mxm ',text)
    text=re.sub(r'([?])\1\1{2,}',r' mqm ',text)
    text=re.sub(r'([*])\1\1{2,}',r'*',text)
    return text


replace_pun = {}
separators = set('"%&\'()+,-./:;<=>@[\\]^_`{|}~')
for punc in separators:
    replace_pun[punc] = ' '
replace_pun['&']=' and '

def my_cleaner(s):
    s = s.lower()
    s=replaceURL(s)
    s=removeUnicode(s)
    s=replaceAbbrev(s)
    
    for punc in separators:
        s = s.replace(punc,replace_pun[punc])               
    return drop_suffix(s)

In [ ]:
df['text'] = df['text'].apply(my_cleaner)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
dataframe_test = pd.read_csv('/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv')
dataframe_test

In [ ]:
dataframe_test = dataframe_test.drop('comment_id', axis = 1)
dataframe_test['text'] = dataframe_test['text'].apply(my_cleaner)


In [ ]:
out = pd.read_csv('/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv')
out


In [ ]:
import io
from tqdm import tqdm
from itertools import islice
import numpy as np

def load_vectors(fname, limit):
  fin = io.open(fname, 'r', encoding = 'utf-8', newline = '\n', errors = 'ignore')
  n, d = map(int, fin.readline().split())
  data = {}
  for line in tqdm(islice(fin, limit), total = limit):
    tokens = line.rstrip().split(' ')
    data[tokens[0]] = np.array(list(map(float, tokens[1:])))
  return data

vecs = load_vectors('../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec', 1300000)   


In [ ]:
from sklearn.linear_model import Ridge
num_iters = 10
y_pred_all = np.zeros((dataframe_test.shape[0], num_iters))
for i in range(num_iters):
    df_test =  pd.concat([df[df['score']>0].sample(frac=0.4, random_state = i*i) , 
                        df[df['score']==0].sample(n=int(len(df[df['score']>0])*0.5) , 
                        random_state = i*i)], axis=0)
    model = Ridge(alpha = 1)
    tfidf =  TfidfVectorizer(min_df= 3, max_df=0.5,analyzer = 'char_wb', ngram_range = (3,5))
    vec = tfidf.fit_transform(df_test['text'])
    model.fit(vec, df_test['score'])
    
    vec_test = tfidf.transform(dataframe_test['text'])
    sample_pred = model.predict(vec_test)
    y_pred_all[:,i] = sample_pred

    
y_pred_tf_idf = np.mean(y_pred_all,axis=1)
mx = y_pred_tf_idf.max()
mn = y_pred_tf_idf.min()
y_pred_tf_idf = (y_pred_tf_idf - mn) / (mx - mn)

In [ ]:
zero = sum(vecs.values()) / len(vecs)
def text2vec(text):
  words = text.split()
  if len(words) == 0:
        return zero
  return sum(list(map(lambda w: np.array(list(vecs.get(w, zero))), words))) / len(words)

In [ ]:
num_iters_w2v = 9
y_pred_w2v = np.zeros((dataframe_test.shape[0], num_iters_w2v))
for i in range(num_iters_w2v):
    print(i)
    df_test =  pd.concat([df[df['score']>0].sample(frac=0.4, random_state = i*i) , 
                        df[df['score']==0].sample(n=int(len(df[df['score']>0])*0.5) , 
                        random_state = i*i)], axis=0)
    model = Ridge(alpha = 1)
    X_train = list(map(lambda text: text2vec(text), df_test['text']))
    model.fit(X_train, df_test['score'])
    
    X_test = list(map(lambda text: text2vec(text), dataframe_test['text']))
    sample_pred = model.predict(X_test)
    y_pred_w2v[:,i] = sample_pred

In [ ]:
y_pred_w2v = np.mean(y_pred_w2v,axis=1)
mx = y_pred_w2v.max()
mn = y_pred_w2v.min()
y_pred_w2v = (y_pred_w2v - mn) / (mx - mn)

In [ ]:
out['score'] = 0.83 * y_pred_tf_idf + 0.17 * y_pred_w2v

In [ ]:
out.to_csv('submission1.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import re 
import scipy
from scipy import sparse

from IPython.display import display
from pprint import pprint
from matplotlib import pyplot as plt 

import time
import scipy.optimize as optimize
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=300
pd.options.display.max_columns = 100

from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn.linear_model import Ridge, Lasso, BayesianRidge
from sklearn.svm import SVR

df_train = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")
df_sub = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")

cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    df_train[category] = df_train[category] * cat_mtpl[category]

df_train['score'] = df_train.loc[:, 'toxic':'identity_hate'].sum(axis=1)

df_train['y'] = df_train['score']

min_len = (df_train['y'] > 0).sum()  # len of toxic comments
df_y0_undersample = df_train[df_train['y'] == 0].sample(n=min_len, random_state=201)  # take non toxic comments
df_train_new = pd.concat([df_train[df_train['y'] > 0], df_y0_undersample])  # make new df
df_train = df_train.rename(columns={'comment_text':'text'})

def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

tqdm.pandas()
df_train['text'] = df_train['text'].progress_apply(text_cleaning)
df = df_train.copy()
df['y'].value_counts(normalize=True)
min_len = (df['y'] >= 0.1).sum()
df_y0_undersample = df[df['y'] == 0].sample(n=min_len * 2, random_state=42)
df = pd.concat([df[df['y'] >= 0.1], df_y0_undersample])
vec = TfidfVectorizer(min_df= 3, max_df=0.8, analyzer = 'char_wb', ngram_range = (3,5))
X = vec.fit_transform(df['text'])
model = Ridge(alpha=0.5)
model.fit(X, df['y'])
l_model = Ridge(alpha=1.)
l_model.fit(X, df['y'])
s_model = Ridge(alpha=2.)
s_model.fit(X, df['y'])
df_val = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
tqdm.pandas()
df_val['less_toxic'] = df_val['less_toxic'].progress_apply(text_cleaning)
df_val['more_toxic'] = df_val['more_toxic'].progress_apply(text_cleaning)
X_less_toxic = vec.transform(df_val['less_toxic'])
X_more_toxic = vec.transform(df_val['more_toxic'])
p1 = model.predict(X_less_toxic)
p2 = model.predict(X_more_toxic)
# Validation Accuracy
print(f'val : {(p1 < p2).mean()}')
df_sub = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
tqdm.pandas()
df_sub['text'] = df_sub['text'].progress_apply(text_cleaning)
X_test = vec.transform(df_sub['text'])
p3 = model.predict(X_test)
p4 = l_model.predict(X_test)
p5 = s_model.predict(X_test)
df_sub['score'] = (p3 + p4 + p5) / 3.
df_sub['score'] = (df_sub['score'] - df_sub['score'].min()) / (df_sub['score'].max() - df_sub['score'].min())
df_sub[['comment_id', 'score']].to_csv("submission2.csv", index=False)

In [ ]:
!pip install pytorch-pretrained-bert


In [ ]:
if True:
    import os
    import gc
    import cv2
    import copy
    import time
    import random

    # For data manipulation
    import numpy as np
    import pandas as pd

    # Pytorch Imports
    import torch
    import torch.nn as nn
    from torch.utils.data import Dataset, DataLoader

    # For Transformer Models
    from transformers import AutoTokenizer, AutoModel,AutoConfig

    # Utils
    from tqdm import tqdm

    # For descriptive error messages
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


    class Config:

        model_name = '../input/roberta-base'

        learning_rate = 3e-4
        epochs = 3
        train_bs =32
        valid_bs = 64
        test_bs = 128

        seed = 2021
        max_length = 128
        min_lr = 1e-7
        scheduler = 'CosineAnnealingLR'
        T_max  = 500
        weight_decay = 1e-6
        max_grad_norm = 1.0
        num_classes = 1
        margin = 0.5
        n_fold = 9
        n_accululate = 1
        device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        hidden_size =768
        num_hidden_layers = 24

        dropout = 0.1


    tokenizer = AutoTokenizer.from_pretrained(Config.model_name)

    MODEL_PATHS = [
        '../input/robertabase5fold2-linear-256/Loss-Fold-0.bin',
        '../input/robertabase5fold2-linear-256/Loss-Fold-1.bin',
        '../input/robertabase5fold2-linear-256/Loss-Fold-2.bin',
        '../input/robertabase5fold2-linear-256/Loss-Fold-3.bin',
        '../input/robertabase5fold2-linear-256/Loss-Fold-4.bin'
    ]

    def set_seed(seed = 42):
        '''Sets the seed of the entire notebook so results are the same every time we run.
        This is for REPRODUCIBILITY.'''
        np.random.seed(seed)
        random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        # When running on the CuDNN backend, two further options must be set
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        # Set a fixed value for the hash seed
        os.environ['PYTHONHASHSEED'] = str(seed)

    set_seed(Config.seed)


    df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
    df['text'] = df['text'].apply(my_cleaner)
    df.head()


    class JigsawDataset(Dataset):
        def __init__(self, df, tokenizer, max_length):
            self.df = df
            self.max_len = max_length
            self.tokenizer = tokenizer
            self.text = df['text'].values

        def __len__(self):
            return len(self.df)

        def __getitem__(self, index):
            text = self.text[index]
            inputs = self.tokenizer.encode_plus(
                            text,
                            truncation=True,
                            add_special_tokens=True,
                            max_length=self.max_len,
                            padding='max_length'
                        )

            ids = inputs['input_ids']
            mask = inputs['attention_mask']        

            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long)
            }


    test_dataset = JigsawDataset(df, tokenizer, max_length=Config.max_length)
    test_loader = DataLoader(test_dataset, batch_size=Config.test_bs, num_workers=2, shuffle=False, pin_memory=True)

    class JModel(nn.Module):
        def __init__(self, checkpoint=Config.model_name, Config=Config):
            super(JModel, self).__init__()
            self.checkpoint = checkpoint
            self.bert = AutoModel.from_pretrained(checkpoint, return_dict=False)
            self.layer_norm = nn.LayerNorm(Config.hidden_size)
            self.dropout = nn.Dropout(Config.dropout)
            self.dense = nn.Sequential(
                nn.Linear(Config.hidden_size, 256),
                nn.LeakyReLU(negative_slope=0.01),
                nn.Dropout(Config.dropout),
                nn.Linear(256, 1)
            )

        def forward(self, input_ids, attention_mask):
            _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = self.layer_norm(pooled_output)
            pooled_output = self.dropout(pooled_output)
            preds = self.dense(pooled_output)
            return preds        


    @torch.no_grad()
    def valid_fn(model, dataloader, device):
        model.eval()

        dataset_size = 0
        running_loss = 0.0

        PREDS = []

        bar = tqdm(enumerate(dataloader), total=len(dataloader))
        for step, data in bar:
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)

            outputs = model(ids, mask)
            PREDS.append(outputs.view(-1).cpu().detach().numpy()) 

        PREDS = np.concatenate(PREDS)
        gc.collect()

        return PREDS

    def inference(model_paths, dataloader, device):
        final_preds = []
        for i, path in enumerate(model_paths):
            model = JModel(Config.model_name)
            model.to(Config.device)
            model.load_state_dict(torch.load(path))

            print(f"Getting predictions for model {i+1}")
            preds = valid_fn(model, dataloader, device)
            final_preds.append(preds)

        final_preds = np.array(final_preds)
        final_preds = np.mean(final_preds, axis=0)
        return final_preds    

    preds = inference(MODEL_PATHS, test_loader, Config.device)    
    df['score'] = preds
    df['score'] = df['score'].rank(method='first')
    df.drop('text', axis=1, inplace=True)
    df.to_csv("submission_bert.csv", index=False)

In [ ]:
import numpy as np
import pandas as pd
import string

import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from wordcloud import STOPWORDS

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('../input/toxic-comments-train/train.csv')
test = pd.read_csv('../input/jigsaw-toxic-severity-rating/validation_data.csv')
sample = pd.read_csv('../input/jigsaw-toxic-severity-rating/sample_submission.csv')
target = pd.read_csv('../input/jigsaw-toxic-severity-rating/comments_to_score.csv')

In [ ]:
train['y'] = train[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].sum(axis=1) > 0
train.drop(['toxic','severe_toxic','obscene','threat','insult','identity_hate'], inplace=True, axis=1)

In [ ]:
count_of_toxic_comments =  train[train.y != 0].shape[0]
count_of_toxic_comments

In [ ]:
train_toxic = train[train.y != 0]
train_non_toxic = train[train.y == 0].sample(count_of_toxic_comments)

In [ ]:
df = pd.concat([train_toxic, train_non_toxic])
df

In [ ]:
df['word_count'] = df['comment_text'].apply(lambda x: len(str(x).split()))

df['unique_word_count'] = df['comment_text'].apply(lambda x: len(set(str(x).split())))

df['stop_word_count'] = df['comment_text'].apply(lambda x: len([w for w in str(x).lower().split() if w in STOPWORDS]))

df['mean_word_length'] = df['comment_text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

df['char_count'] = df['comment_text'].apply(lambda x: len(str(x)))

df['punctuation_count'] = df['comment_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

In [ ]:
METAFEATURES = ['word_count', 'unique_word_count', 'stop_word_count', 'mean_word_length','char_count', 'punctuation_count']
TOXIC_COMMENTS = df['y'] == 1

fig, axes = plt.subplots(ncols=2, nrows=len(METAFEATURES), figsize=(20, 50), dpi=100)

for i, feature in enumerate(METAFEATURES):
    sns.distplot(df.loc[~TOXIC_COMMENTS][feature], label='Non Toxic', ax=axes[i][0], color='green')
    sns.distplot(df.loc[TOXIC_COMMENTS][feature], label='Toxic', ax=axes[i][0], color='red')

    sns.distplot(df[feature], label='Train', ax=axes[i][1])
    
    for j in range(2):
        axes[i][j].set_xlabel('')
        axes[i][j].tick_params(axis='x', labelsize=12)
        axes[i][j].tick_params(axis='y', labelsize=12)
        axes[i][j].legend()
    
    axes[i][0].set_title(f'{feature} Target Distribution in Training Set', fontsize=13)

plt.show()

In [ ]:
# Remove stopwords & convert to lower case
df['comment_text'] = df['comment_text'].apply(lambda x: ' '.join([w for w in str(x).lower().split() if w not in STOPWORDS]))

# Remove Punctuations
df["comment_text"] = df['comment_text'].str.replace('[^\w\s]','')
df.tail()

In [ ]:
X = df.drop(['y'], axis=1)
y = df['y']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [ ]:
X_train = X_train.comment_text.values
X_test = X_test.comment_text.values

In [ ]:
OOV_TOKEN = '<OOV>'
VOCAB_SIZE = 10000
MAX_LEN = 100
EMBEDDING_DIM = 100

In [ ]:
tokenizer = Tokenizer(
    num_words=VOCAB_SIZE,
    oov_token=OOV_TOKEN
)
tokenizer.fit_on_texts(X_train)

In [ ]:
train_seq = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(
    train_seq, maxlen=MAX_LEN, dtype='int32', padding='post',
    truncating='post'
)

test_seq = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(
    test_seq, maxlen=MAX_LEN, dtype='int32', padding='post',
    truncating='post'
)

In [ ]:
model = tf.keras.Sequential([
  Embedding(VOCAB_SIZE, EMBEDDING_DIM, name="embedding"),
    LSTM(64),
    Dropout(0.2),
  Dense(128, activation='relu'),
    Dropout(0.1),
  Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
es = EarlyStopping(patience=15, 
                   monitor='loss', 
                   restore_best_weights=True, 
                   mode='min', 
                   verbose=1)

In [ ]:
hist = model.fit(
    train_padded,
    y = y_train,
    validation_data=(test_padded, y_test),
    epochs=15,
    callbacks=es
)


In [ ]:
df_target = target


In [ ]:
df_target['text'] = df_target['text'].apply(lambda x: ' '.join([w for w in str(x).lower().split() if w not in STOPWORDS]))

df_target["text"] = df_target['text'].str.replace('[^\w\s]','')
df_target.head()

In [ ]:
target_seq = tokenizer.texts_to_sequences(df_target.text.values)
target_padded = pad_sequences(
    target_seq, maxlen=MAX_LEN, dtype='int32', padding='post',
    truncating='post'
)

In [ ]:
result = model.predict(target_padded)


In [ ]:
sample


In [ ]:
result = result.reshape(-1)
result = (result - result.min()) / (result.max() - result.min())

In [ ]:
brt = pd.read_csv("submission_bert.csv")['score']
brt = (brt - brt.min()) / (brt.max() - brt.min())

In [ ]:
target['score'] = result * 0.1 + brt * 0.9
aue = target['score']

In [ ]:
from scipy.stats import rankdata
#aue=rankdata(aue, method='ordinal')
data = pd.read_csv("./submission1.csv")
data["score1"] = data["score"]
#data["score1"] = rankdata(data["score1"], method = 'ordinal')

data["score2"] = pd.read_csv("./submission2.csv")["score"]
#data["score2"] = rankdata( data["score2"], method='ordinal')

In [ ]:
data['score'] = data['score1'] * 0.5 + data['score2'] * 0.92 + aue * 0.125

In [ ]:
df_test = data

In [ ]:
df_test["score"] = rankdata( df_test["score"], method='ordinal')
df_test

In [ ]:
df_test[['comment_id','score']].to_csv('submission.csv', index = False)

In [ ]:
pd.read_csv("./submission.csv")
